# Appendix 4

In [ ]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)

import luxgiant_clinical.ThreeCatAnalysis as thr
from luxgiant_clinical.Helpers import categories_recoder

import pandas as pd
import numpy as np

In [ ]:
# load data

folder_path = os.path.join(path_to_lib, 'data/source')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [ ]:
# filter data to keep only patients with PD subtype information

mask_patients= (df['Status']=='Patient')
mask_agecat = ~df['age_category'].isnull()

df_cases = df[mask_patients & mask_agecat].reset_index(drop=True)

del df

df_cases = df_cases[~(df_cases['age_category']=='Onset <21 years')].reset_index(drop=True)
df_cases.shape

In [ ]:
df_cases['age_category'].value_counts().reset_index()

In [ ]:
# rename and select first group of variables
variables_dict_1 = {
    "age"                       : "Age at Recruitment (years)", 
    "age_at_onset"              : "Age at Onset (years)" ,      
    "sex"                       : "Male" ,                      
    "PD_duration"               : "Duration of Disease (years)",
    "years_of_education"        : "Years of Education",         
    "bmi_comp"                  : "Body Mass Index",            
    "reason"                    : "PD-related Reason for Stop Working", 
    "work"                      : "Currently Employed",         
    "nature_of_work___1"        : "Agriculture as a job",     
    "over_your_lifetime_have_yo": "Life time direct exposure to pesticide/insecticide/fungicide",
    "in_your_lifetime_have_you" : "Smoked 100 or more cigarettes in lifetime",
    "in_your_lifetime_have"     : "Regular consumption of caffeinated drinks for >6months",
    "have_you_ever_had_a_head_i": "Head injury or concussion",

}
variables_1 = ['participant_id', 'age_category'] + list(variables_dict_1.keys())

# statistical measures
stats_meas = {
    'mean'  : ["Age at Recruitment (years)", "Age at Onset (years)", "Body Mass Index"],
    'n'     : ["Male", "Currently Employed", "PD-related Reason for Stop Working", "Agriculture as a job", 
               "Life time direct exposure to pesticide/insecticide/fungicide", 
               "Smoked 100 or more cigarettes in lifetime", "Regular consumption of caffeinated drinks for >6months",
               "Head injury or concussion"],
    'median': ["Duration of Disease (years)", "Years of Education"]
}

groups = ['Onset 21-49 years', 'Onset 50-60 years', 'Onset >60 years']

In [ ]:
# keep only variables for analysis

df_cases_1 = df_cases[variables_1].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict_1)

In [ ]:
# recode variable values

recode_dict = {
    "Currently Employed"                :{'Yes':1, 'No':0},
    "Agriculture as a job"              :{'Checked':1, 'Unchecked':0},
    "PD-related Reason for Stop Working":{'PD related':1, 'not PD related':0},
    "Male"                              :{'Male':1, 'Female':0}
}
for key in recode_dict.keys():
    df_cases_1 = categories_recoder(df_cases_1, [key], recode_dict[key])

recode = ["Life time direct exposure to pesticide/insecticide/fungicide", 
          "Smoked 100 or more cigarettes in lifetime",
          "Regular consumption of caffeinated drinks for >6months", "Head injury or concussion"]
mapping = {'Yes':1, 'No':0, 'Dont Know':np.nan, 'Refused':np.nan, 'Possibly':np.nan}

df_cases_1 = categories_recoder(df_cases_1, recode, mapping)

In [ ]:
summary = [
    thr.report_mean_std(
        data_df    =df_cases_1,
        variables  =stats_meas['mean'],
        groups     =groups,
        grouping_by='age_category'
    ),
    thr.report_median_iqr(
        data_df    =df_cases_1,
        variables  =stats_meas['median'],
        groups     =groups,
        grouping_by='age_category'
    ),
    thr.report_proportion(
        data_df    =df_cases_1,
        variables  =stats_meas['n'],
        groups     =groups,
        grouping_by='age_category'
    )
]

In [ ]:
# rename and select second group of variables

variables_dict_2 = {
    "medical_history_neurologic___7": "Stroke",
    "medical_history_cancer___1"    : "Melanoma" ,
    "medical_history_cancer___2"    : "Prostate Cancer" ,
    "medical_history_cancer___3"    : "Other Cancers",
    "medical_history_metabolic___1" : "Diabetes",
    "medical_history_metabolic___3" : "High Cholesterol",
    "medical_history_cardiovasc___2": "Hypertension",
    "medical_history_cardiovasc___3": "History of CAD",
    "medical_history_psychologi___1": "History of Anxiety",
    "medical_history_psychologi___3": "History of Depression",
    "family_member_diagnosed_wi"    : "Family History of PD",
    "family_member_diagnosed_wi_2"  : "Family History of Tremor",
    "family_member_diagnosed"       : "Family History of Dementia"
}
variables_2 = ['participant_id', 'age_category'] + list(variables_dict_2.keys())

# statistical measures
stats_meas2 = {
    'n'     : ["Stroke", "Melanoma", "Prostate Cancer", "Other Cancers", "Diabetes", "High Cholesterol", "Hypertension",
               "History of CAD", "History of Anxiety", "History of Depression", "Family History of PD", "Family History of Tremor", "Family History of Dementia"]
}

In [ ]:
# keep only variables for analysis

df_cases_2 = df_cases[variables_2].copy()
df_cases_2 = df_cases_2.rename(columns=variables_dict_2)

In [ ]:
# recode variable values

fam_hist = ["Family History of PD", "Family History of Tremor", "Family History of Dementia"]
remaining= [feat for feat in stats_meas2['n'] if feat not in fam_hist]

df_cases_2 = categories_recoder(df_cases_2, fam_hist, {"Yes":1, "No":0})
df_cases_2 = categories_recoder(df_cases_2, remaining, {"Checked":1, "Unchecked":0})

In [ ]:
# count and proportion of categorical variables

df_sum4 = thr.report_proportion(
    data_df    =df_cases_2,
    variables  =stats_meas2['n'],
    groups     =['Onset 21-49 years', 'Onset 50-60 years', 'Onset >60 years'],
    grouping_by='age_category',
    subheader  ='Medical History'
)

In [ ]:
# concatenate all results

df_11 = pd.concat(
    summary + [df_sum4], axis=0,ignore_index=True
)

In [ ]:
df_cases_3 = df_cases_1[~(df_cases_1['age_category']=='Onset 50-60 years')].reset_index(drop=True)
df_cases_4 = df_cases_2[~(df_cases_2['age_category']=='Onset 50-60 years')].reset_index(drop=True)

In [ ]:
summary1 = [
    thr.bonferroni_mean_std(
        data_df      =df_cases_3, 
        variables    =stats_meas['mean'], 
        groups       =['Onset 21-49 years', 'Onset >60 years'],
        grouping_by  ='age_category',
        correc_factor=3
    ),
    thr.bonferroni_median_iqr(
        data_df      =df_cases_3, 
        variables    =stats_meas['median'], 
        groups       =['Onset 21-49 years', 'Onset >60 years'],
        grouping_by  ='age_category',
        correc_factor=3
    ),
    thr.bonferroni_proportions(
        data_df      =df_cases_3, 
        variables    =stats_meas['n'], 
        groups       =['Onset 21-49 years', 'Onset >60 years'],
        grouping_by  ='age_category',
        correc_factor=3
    ),
    thr.bonferroni_proportions(
        data_df    =df_cases_4,
        variables  =stats_meas2['n'],
        groups     =['Onset 21-49 years', 'Onset >60 years'],
        grouping_by='age_category',
        correc_factor=3,
        subheader  ='Medical History'
    )
]

# concatenate all results
df_111 = pd.concat(
    summary1, axis=0,ignore_index=True
)
df_111.columns = ['Variable', 'Adjusted p-value (Early vs Late)']

In [ ]:
df_cases_5 = df_cases_1[~(df_cases_1['age_category']=='Onset >60 years')].reset_index(drop=True)
df_cases_6 = df_cases_2[~(df_cases_2['age_category']=='Onset >60 years')].reset_index(drop=True)

In [ ]:
summary2 = [
    thr.bonferroni_mean_std(
        data_df      =df_cases_5, 
        variables    =stats_meas['mean'], 
        groups       =['Onset 21-49 years', 'Onset 50-60 years'],
        grouping_by  ='age_category',
        correc_factor=3
    ),
    thr.bonferroni_median_iqr(
        data_df      =df_cases_5, 
        variables    =stats_meas['median'], 
        groups       =['Onset 21-49 years', 'Onset 50-60 years'],
        grouping_by  ='age_category',
        correc_factor=3
    ),
    thr.bonferroni_proportions(
        data_df      =df_cases_5, 
        variables    =stats_meas['n'], 
        groups       =['Onset 21-49 years', 'Onset 50-60 years'],
        grouping_by  ='age_category',
        correc_factor=3
    ),
    thr.bonferroni_proportions(
        data_df    =df_cases_6,
        variables  =stats_meas2['n'],
        groups     =['Onset 21-49 years', 'Onset 50-60 years'],
        grouping_by='age_category',
        correc_factor=3,
        subheader  ='Medical History'
    )
]

# concatenate all results
df_112 = pd.concat(
    summary2, axis=0,ignore_index=True
)
df_112.columns = ['Variable', 'Adjusted p-value (Early vs Medium)']

In [ ]:
df_cases_7 = df_cases_1[~(df_cases_1['age_category']=='Onset 21-49 years')].reset_index(drop=True)
df_cases_8 = df_cases_2[~(df_cases_2['age_category']=='Onset 21-49 years')].reset_index(drop=True)

In [ ]:
summary3 = [
    thr.bonferroni_mean_std(
        data_df      =df_cases_7, 
        variables    =stats_meas['mean'], 
        groups       =['Onset 50-60 years', 'Onset >60 years'],
        grouping_by  ='age_category',
        correc_factor=3
    ),
    thr.bonferroni_median_iqr(
        data_df      =df_cases_7, 
        variables    =stats_meas['median'], 
        groups       =['Onset 50-60 years', 'Onset >60 years'],
        grouping_by  ='age_category',
        correc_factor=3
    ),
    thr.bonferroni_proportions(
        data_df      =df_cases_7, 
        variables    =stats_meas['n'], 
        groups       =['Onset 50-60 years', 'Onset >60 years'],
        grouping_by  ='age_category',
        correc_factor=3
    ),
    thr.bonferroni_proportions(
        data_df    =df_cases_8,
        variables  =stats_meas2['n'],
        groups     =['Onset 50-60 years', 'Onset >60 years'],
        grouping_by='age_category',
        correc_factor=3,
        subheader  ='Medical History'
    )
]

# concatenate all results
df_113 = pd.concat(
    summary3, axis=0, ignore_index=True
)
df_113.columns = ['Variable', 'Adjusted p-value (Medium vs Late)']

In [ ]:
df_11 = thr.final_formatter(
    overall_df=df_11,
    adjusted_df=[df_111, df_112, df_113],
    groups=groups
)

In [ ]:
# save result in csv file

df_11.to_csv(os.path.join(path_to_lib, 'data/auxiliar/appendix_4.csv'), index=False)
df_11